Import required libraries

In [2]:
import numpy as np
import pymc as pm
import pandas as pd
import subprocess
import matplotlib.pyplot as plt

# Load observed data (replace with actual Y_observed from your black-box model)
Y_observed = pd.read_csv("Y_out.csv", header=None).values.flatten()

# Function to run the black-box model and get the output
def black_box_likelihood(X_a):
    """
    Calls the external black-box model to generate Y given input X_a.
    Returns the generated output Y.
    """
    # Convert PyMC tensor to NumPy array for saving (evaluate symbolic variables)
    X_a_eval = np.array([X_a[i].eval() for i in range(3)]).reshape(1, -1)

    # Save input to a temporary file
    input_file = 'input_temp.txt'
    np.savetxt(input_file, X_a_eval, delimiter=',')

    # Run the black-box model
    subprocess.run(['./local_model_windows.exe', input_file], capture_output=True, text=True)

    # Load the generated output
    output_file = 'Y_out.csv'
    Y_generated = pd.read_csv(output_file, header=None).values.flatten()  # Ensure correct shape

    return Y_generated

# Bayesian Inference Model using PyMC
with pm.Model() as model:
    # Define priors for the three aleatory input variables (assumed uniform between 0 and 1)
    X_a1 = pm.Uniform('X_a1', lower=0, upper=1)
    X_a2 = pm.Uniform('X_a2', lower=0, upper=1)
    X_a3 = pm.Uniform('X_a3', lower=0, upper=1)

    # Stack the inputs into a single tensor
    X_a = pm.math.stack([X_a1, X_a2, X_a3])

    # Likelihood: Use the black-box model to get Y given X_a
    sigma = pm.HalfNormal('sigma', sigma=1)  # Noise in the outputs
    Y_obs = pm.Normal('Y_obs', mu=black_box_likelihood(X_a), sigma=sigma, observed=Y_observed)

    # Perform MCMC sampling
    trace = pm.sample(10, tune=2, chains=2, return_inferencedata=True)

# Plot posterior distributions of the aleatory variables
pm.plot_posterior(trace, var_names=['X_a1', 'X_a2', 'X_a3'])
plt.show()


Only 10 samples per chain. Reliable r-hat and ESS diagnostics require longer chains for accurate estimate.


KeyboardInterrupt: 

Define model inputs

In [2]:
# Step 1: Define input parameters to the model
# Define the input vector X_input with the required values
X_input = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.1, 0.1, 0.0, 42])
# Alternatively, the model also supports a batch of N input vectors (as an example, "N" copies of X_input)
N = 3
X_input_batch = np.tile(X_input, (N, 1))  # Shape will be (N, 9)

input_file_path = 'input.txt'
np.savetxt(input_file_path, X_input_batch, delimiter=',')
print(f'Input data written to {input_file_path}')

# Step 3: Run the executable and capture its output
exe_path = os.path.abspath(Path(os.getcwd()) / "local_model_windows.exe")  # Path to the executable in the current folder
command = [exe_path, input_file_path]

print('Simulation executable has been called.')
result = subprocess.run(command, capture_output=True, text=True)


# Print the output from the executable
print(result.stdout)

Input data written to input.txt
Simulation executable has been called.



Save inputs to a file

Input data written to input.txt
Simulation executable has been called.



Run the local blackbox model

Simulation executable has been called.



Write Output to a .csv file

In [5]:
# Step 1: Load the output data
# Load the output data from the CSV file into a pandas DataFrame
output_file_path = 'Y_out.csv'
df = pd.read_csv(output_file_path, header=None)
print(f'Simulation output data loaded from {output_file_path}')

# Step 2: Extract unique sample indices (number "N" if using batch input) and remove that column (Column 7 contains sample indices)
sample_indices = df[6].unique()  # Extract unique sample indices
num_samples = len(sample_indices)
df = df.drop(columns=[6])  # Drop the sample index column

# Step 3: Reshape the Output Data (6 features, 60 timesteps per simulation)
# Convert DataFrame to a NumPy array and reshape/transpose to the desired 3D shape (num_time_steps x num features x num_samples)
Y_out = df.to_numpy().reshape(num_samples, 60, 6).transpose(1, 2, 0)

# Step 4: As an example, Print the First Time Step for All 6 Output Features of Sample 3
print('Output values for the first timestep for all 6 features of sample 3:')
print(Y_out[0, :, 2])

Simulation output data loaded from Y_out.csv
Output values for the first timestep for all 6 features of sample 3:
[0. 0. 0. 0. 0. 0.]


In [6]:
# Bayesian inference
with pm.Model() as model:
    # Priors for aleatory input variables (X_a1, X_a2, X_a3)
    X_a1 = pm.Uniform('X_a1', lower=0, upper=1)
    X_a2 = pm.Uniform('X_a2', lower=0, upper=1)
    X_a3 = pm.Uniform('X_a3', lower=0, upper=1)


def black_box_likelihood(X_a):
    # Load the generated output
        Y_generated = pd.read_csv('Y_out.csv', header=None).values

        return Y_generated.flatten()  # Ensure correct shape


sigma = pm.HalfNormal('sigma', sigma=1)  # Noise in the outputs
Y_obs = pm.Normal('Y_obs', mu=black_box_likelihood(X_a), sigma=sigma, observed=Y_observed.flatten())

# Perform MCMC sampling
trace = pm.sample(2000, tune=1000, chains=2, return_inferencedata=True)

TypeError: No model on context stack, which is needed to instantiate distributions. Add variable inside a 'with model:' block, or use the '.dist' syntax for a standalone distribution.